In [5]:
from typing import TypedDict, Annotated
from dotenv import load_dotenv
from langchain.chat_models import init_chat_model
from langgraph.graph import StateGraph, START, END, add_messages


class State(TypedDict) :
    messages: Annotated[list, add_messages]

load_dotenv()

llm = init_chat_model("google_genai:gemini-2.0-flash")

def chatbot(state: State) -> State :
    return {"messages": [llm.invoke(state["messages"])]}

builder = StateGraph(State)

builder.add_node("chatbot_node", chatbot)

builder.add_edge(START, "chatbot_node")
builder.add_edge("chatbot_node", END)

# call builder to build graph
graph = builder.compile()

state = None

while True:
    in_message = input("You:  ")
    if in_message.lower() in ["exit", "quit"]:
        break
    if state is None:
        state: State = {
            "messages": [{"role": "user", "content": in_message}],
        }
    else:
        state["messages"].append({"role": "user", "content": in_message})

    state = graph.invoke(state)
    print(state)
    print("Bot: ", state["messages"][-1].content)




{'messages': [HumanMessage(content='What is the latest price of MFST Stock', additional_kwargs={}, response_metadata={}, id='7276e3ea-23e7-4a71-aa53-79fbe55ea86b'), AIMessage(content='Unfortunately, I cannot provide you with real-time stock prices. The price of MSFT (Microsoft) stock fluctuates throughout the day.\n\nTo get the latest, up-to-the-minute price, I recommend checking these resources:\n\n*   **Google Finance:** Search "MSFT stock" on Google.\n*   **Yahoo Finance:** finance.yahoo.com\n*   **Your Brokerage Account:** If you have a brokerage account (e.g., Fidelity, Schwab, Robinhood), the most accurate price will be available there.\n\nThese sources will give you the most current information.', additional_kwargs={}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'model_name': 'gemini-2.0-flash', 'safety_ratings': []}, id='run--ed66fa46-2a34-477e-bed5-7594dc80e405-0', usage_metadata={'input_tokens': 9, 'output_tokens':